# Simple Neural Network Assignment
## Large Language Models

Resources:

1. https://pytorch.org/vision/stable/generated/torchvision.datasets.MNIST.html
2. https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

Create a Conda environment for installations:

1. https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#activating-an-environment

In [16]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# dataset import 
from torchvision.datasets import MNIST
import torchvision.transforms as transforms

### Download and Transform the Data

In [20]:
# Define transformations for the training set, flip the images randomly, crop out and apply mean and std normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Download and load the training data
trainset = MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data
testset = MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

### Definte the NN

In [11]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using mps device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### Define Train and Test Procedures

In [12]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [13]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [14]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

### Train using the MNIST Dataset

In [21]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(trainloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.388608  [   64/60000]
loss: 0.351339  [ 6464/60000]
loss: 0.276522  [12864/60000]
loss: 0.591416  [19264/60000]
loss: 0.376442  [25664/60000]
loss: 0.429799  [32064/60000]
loss: 0.377038  [38464/60000]
loss: 0.422995  [44864/60000]
loss: 0.489741  [51264/60000]
loss: 0.554684  [57664/60000]
Test Error: 
 Accuracy: 88.9%, Avg loss: 0.409299 

Epoch 2
-------------------------------
loss: 0.492134  [   64/60000]
loss: 0.308400  [ 6464/60000]
loss: 0.552904  [12864/60000]
loss: 0.281313  [19264/60000]
loss: 0.319501  [25664/60000]
loss: 0.393553  [32064/60000]
loss: 0.437990  [38464/60000]
loss: 0.445481  [44864/60000]
loss: 0.490780  [51264/60000]
loss: 0.333147  [57664/60000]
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.381104 

Epoch 3
-------------------------------
loss: 0.476674  [   64/60000]
loss: 0.369592  [ 6464/60000]
loss: 0.598428  [12864/60000]
loss: 0.459573  [19264/60000]
loss: 0.195318  [25664/60000]
loss: 0.354528  [32064/600